# Решение задачи предсказания дефолта

In [1]:
# Загрузка данных с kaggle. Нужно загрузить kaggle.json с профиля в Kaggle.
!mkdir -p ~/.kaggle
!mv kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json
!kaggle competitions download -c school-of-quants-hackathon-2025-finals
!unzip school-of-quants-hackathon-2025-finals.zip

  0% 0.00/43.2M [00:00<?, ?B/s]
100% 43.2M/43.2M [00:00<00:00, 1.10GB/s]
Archive:  school-of-quants-hackathon-2025-finals.zip
  inflating: X_test.csv              
  inflating: X_train.csv             
  inflating: y_train.csv             


In [212]:
import os
import pandas as pd
import numpy as np
from sklearn.model_selection import StratifiedKFold, cross_val_score
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder, OrdinalEncoder, StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import roc_auc_score, f1_score
from sklearn.model_selection import train_test_split
import joblib
from tqdm import tqdm

In [245]:
X_test

,id,credit_number_for_user,days_since_confirmed,maturity_plan,maturity_fact,credit_limit,next_payment_sum,sum_left_to_pay,current_overdue_debt,max_overdue_debt,...,enc_paym_17,enc_paym_18,enc_paym_19,enc_paym_20,enc_paym_21,enc_paym_22,enc_paym_23,enc_paym_24,credit_type,credit_currency
0,532674,5,8,15,9,16,2,3,0,2,...,3,3,3,4,3,3,3,4,4,1
1,1048835,3,13,13,0,18,2,3,0,2,...,3,3,3,4,3,3,3,4,4,1
2,184294,4,12,8,6,12,2,3,0,2,...,3,3,3,4,3,3,3,4,4,1
3,1075748,5,12,1,16,0,2,3,0,2,...,0,0,0,1,0,0,0,1,4,1
4,2034965,12,6,4,8,8,3,3,0,2,...,3,3,3,4,3,3,3,4,3,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
456847,1502480,6,14,9,0,13,2,3,0,2,...,3,3,3,4,3,3,3,4,4,1
456848,1469413,10,3,15,9,17,2,3,0,2,...,3,3,3,4,3,3,3,4,4,1
456849,263710,2,5,17,8,13,2,3,0,2,...,3,3,3,4,3,3,3,4,3,1
456850,1119056,16,9,4,8,8,4,2,0,2,...,3,3,3,4,3,3,3,4,3,1


In [213]:
N_rows = None # None чтобы загрузить все
X_train = pd.read_csv('X_train.csv', nrows=N_rows)
y_train = pd.read_csv('y_train.csv', nrows=N_rows)
X_test = pd.read_csv('X_test.csv')
print('X_train.shape =', X_train.shape)
print('y_train.shape =', y_train.shape)
print('X_test.shape  =', X_test.shape)
X_train.head()

X_train.shape = (1827404, 48)
y_train.shape = (1827404, 2)
X_test.shape  = (456852, 48)


,id,credit_number_for_user,days_since_confirmed,maturity_plan,maturity_fact,credit_limit,next_payment_sum,sum_left_to_pay,current_overdue_debt,max_overdue_debt,...,enc_paym_17,enc_paym_18,enc_paym_19,enc_paym_20,enc_paym_21,enc_paym_22,enc_paym_23,enc_paym_24,credit_type,credit_currency
0,1329062,2,14,9,12,5,2,3,0,2,...,3,3,3,4,3,3,3,4,3,1
1,747472,4,3,1,10,14,2,3,0,2,...,0,0,0,4,3,3,3,4,4,1
2,2217455,12,10,1,8,0,5,1,0,2,...,3,3,3,4,3,3,3,4,4,1
3,1103371,7,17,8,5,6,2,3,0,2,...,3,3,3,4,3,3,3,4,4,1
4,517339,16,10,4,8,14,2,3,0,2,...,3,3,3,4,3,3,3,4,3,1


In [214]:
y_train.head()

,id,flag
0,1329062,False
1,747472,False
2,2217455,False
3,1103371,True
4,517339,False


In [215]:
y_train['flag'].value_counts()

,count
flag,
False,1767440
True,59964


In [216]:
from imblearn.under_sampling import RandomUnderSampler
import pandas as pd

# X_train — признаки
# y_train — DataFrame с колонками 'id' и 'flag'

# Берём только флаг для undersampling
y_labels = y_train['flag']

# Создаём undersampler
rus = RandomUnderSampler(random_state=42)
X_res, y_res = rus.fit_resample(X_train, y_labels)

# Получаем индексы выбранных строк
selected_idx = rus.sample_indices_ if hasattr(rus, 'sample_indices_') else rus.sample_indices_

# Восстанавливаем X_train
X_train = pd.DataFrame(X_res, columns=X_train.columns)

# Восстанавливаем y_train с id
y_train = y_train.iloc[selected_idx].copy()
y_train['flag'] = y_res.values
y = y_train['flag'].values

print(y_train.head())
print(y_train['flag'].value_counts())

              id   flag
1687980  2205711  False
78283     141866  False
234418    753302  False
1736807   325976  False
1765493  2223380  False
flag
False    59964
True     59964
Name: count, dtype: int64


## Обработка данных

In [217]:
import numpy as np
import pandas as pd
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, OneHotEncoder, FunctionTransformer
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import StratifiedKFold, cross_val_score
from sklearn.metrics import roc_auc_score

In [218]:
EPS = 1e-9

# ---------- Автоматическое выделение групп колонок ----------
all_cols = X_train.columns.tolist()
id_col = 'id'

# ручные имена (подправьте если у вас другие)
num_base = [
    'credit_number_for_user', 'days_since_confirmed',
    'maturity_plan','maturity_fact',
    'credit_limit','next_payment_sum','sum_left_to_pay',
    'current_overdue_debt','max_overdue_debt','full_credit_cost'
]
# проверяем какие из них реально есть
num_cols = [c for c in num_base if c in all_cols]

assert len(num_cols) == len(num_base)

overdues_cols = [c for c in all_cols if c.startswith('overdues_')]
no_overdues_cols = [c for c in all_cols if c.startswith('no_overdues_')]
enc_paym_cols = [c for c in all_cols if c.startswith('enc_paym_')]

cat_cols = [c for c in ['credit_type','credit_currency'] if c in all_cols]

In [219]:
from sklearn.base import BaseEstimator, TransformerMixin
import numpy as np
import pandas as pd

# ---------- Кастомный трансформер: фичи из enc_paym_ последовательности ----------
class EncPaymFeatures(BaseEstimator, TransformerMixin):
    def __init__(self, cols):
        self.cols = cols
        self.cols_out = ['enc_first', 'enc_last', 'enc_mean', 'enc_std', 'enc_slope',
                         'enc_n_unique', 'enc_missing_frac', 'enc_num_changes']

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        out = []
        for _, row in X[self.cols].iterrows():
            vals = row.values.astype(float)
            mask = ~pd.isnull(vals)
            if mask.sum() == 0:
                feats = [np.nan]*len(self.cols_out)
            else:
                v = vals[mask]
                first = v[0]
                last = v[-1]
                mean = float(np.mean(v))
                std = float(np.std(v))
                n_unique = len(np.unique(v))
                missing_frac = 1.0 - (mask.sum() / len(self.cols))
                if len(v) >= 2:
                    xs = np.arange(len(v))
                    slope = np.polyfit(xs, v, 1)[0]
                else:
                    slope = 0.0
                num_changes = int(np.sum(np.diff(v) != 0))
                feats = [first, last, mean, std, slope, n_unique, missing_frac, num_changes]
            out.append(feats)
        return np.array(out)

    def get_feature_names_out(self, input_features=None):
        return self.cols_out


# ---------- Кастомный трансформер: агрегаты по overdues / no_overdues ----------
class OverduesFeatures(BaseEstimator, TransformerMixin):
    def __init__(self, over_cols, no_over_cols):
        self.over_cols = over_cols
        self.no_over_cols = no_over_cols
        self.cols_out = ['total_over', 'max_bucket_idx', 'num_over_buckets_nonzero', 'total_no_over', 'frac_no_over']

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        over_cols = [c for c in self.over_cols if c in X.columns]
        no_over_cols = [c for c in self.no_over_cols if c in X.columns]
        out = []
        for _, row in X.iterrows():
            over = row[over_cols].fillna(0).values if len(over_cols) > 0 else np.array([0.])
            no_over = row[no_over_cols].fillna(0).values if len(no_over_cols) > 0 else np.array([0.])
            total_over = np.sum(over)
            total_no_over = np.sum(no_over)
            max_bucket_idx = int(np.argmax(over)) if len(over) > 0 else -1
            num_over_buckets_nonzero = int(np.sum(over > 0))
            frac_no_over = float(total_no_over / (len(no_over_cols) + 1e-8)) if len(no_over_cols) > 0 else 0.0
            out.append([total_over, max_bucket_idx, num_over_buckets_nonzero, total_no_over, frac_no_over])
        return np.array(out)

    def get_feature_names_out(self, input_features=None):
        return self.cols_out

In [220]:
# ---------- Фичеринжиниринг для основных числовых колонок ----------
def num_feature_engineering(df):
    df = df.copy()
    # даты/планы
    if 'maturity_plan' in df.columns and 'maturity_fact' in df.columns:
        df['maturity_diff'] = df['maturity_fact'] - df['maturity_plan']
        df['maturity_ratio'] = df['maturity_fact'] / (df['maturity_plan'] + EPS)
        df['closed_early'] = (df['maturity_fact'] < df['maturity_plan']).astype(int)
    # utilization
    if 'sum_left_to_pay' in df.columns and 'credit_limit' in df.columns:
        df['utilization'] = df['sum_left_to_pay'] / (df['credit_limit'] + EPS)
    # payment ratios
    if 'next_payment_sum' in df.columns and 'credit_limit' in df.columns:
        df['next_payment_ratio'] = df['next_payment_sum'] / (df['credit_limit'] + EPS)
    # current overdue flag
    if 'current_overdue_debt' in df.columns:
        df['has_current_overdue'] = (df['current_overdue_debt'] > 0).astype(int)
    # log transforms for skewed money cols
    for c in ['credit_limit','next_payment_sum','sum_left_to_pay','current_overdue_debt','max_overdue_debt','full_credit_cost']:
        if c in df.columns:
            df[c+'_log1p'] = np.log1p(df[c].fillna(0.0).astype(float))
    # days_since_confirmed clip
    if 'days_since_confirmed' in df.columns:
        df['days_since_confirmed'] = df['days_since_confirmed'].clip(lower=0)
    return df

In [221]:
# ---------- Применяем фичеринжиниринг к train/test ----------
X_train_fe = num_feature_engineering(X_train)
X_test_fe = num_feature_engineering(X_test)

# обновляем списки колонок (новые фичи)
# numeric features for pipeline
numeric_engineered = [c for c in X_train_fe.columns if c not in ([id_col] + enc_paym_cols + overdues_cols + no_overdues_cols + cat_cols)]
# remove original enc_paym and overdues/no_overdues from numeric list if present
numeric_engineered = [c for c in numeric_engineered if not c.startswith('enc_paym_') and not c.startswith('overdues_') and not c.startswith('no_overdues_')]

# ---------- ColumnTransformer ----------
num_imputer = SimpleImputer(strategy='median')
num_pipeline = Pipeline([
    ('imputer', num_imputer),
    ('scaler', StandardScaler()),   # если будете использовать дерево, можно убрать scaler
])

cat_pipeline = Pipeline([
    ('imputer', SimpleImputer(strategy='constant', fill_value=0)),
    ('ohe', OneHotEncoder(handle_unknown='ignore', sparse_output=False))
])

preprocessor = ColumnTransformer(
    transformers=[
        ('num', num_pipeline, numeric_engineered),
        ('enc_paym', EncPaymFeatures(enc_paym_cols), enc_paym_cols if len(enc_paym_cols)>0 else []),
        ('overdue', OverduesFeatures(overdues_cols, no_overdues_cols), overdues_cols + no_overdues_cols if (len(overdues_cols)>0 or len(no_overdues_cols)>0) else []),
        ('cat', cat_pipeline, cat_cols),
    ],
    remainder='passthrough'
)

## Модель LGM

In [222]:
X_train_transformed = preprocessor.fit_transform(X_train_fe, y)

In [223]:
X_train_transformed_df = pd.DataFrame(
    X_train_transformed,
    columns=preprocessor.get_feature_names_out()
)

In [227]:
id_col = 'remainder__id'

In [228]:
X_train_transformed_df.shape, y.shape

((119928, 47), (119928,))

In [229]:
Xtr, Xte, ytr, yte = train_test_split(X_train_transformed_df, y, test_size=0.2)

In [234]:
import lightgbm as lgb

X_train_lgb = Xtr.drop(id_col, axis=1)
train_data = lgb.Dataset(X_train_lgb, label=ytr)

params = {
    'objective': 'binary',
    'metric': 'f1',
    'boosting_type': 'gbdt',
    'learning_rate': 0.05,
    'num_leaves': 63,
    'n_jobs': -1,
    'seed': 42,
    'verbose': 1
}

In [235]:
model = lgb.train(params, train_data, num_boost_round=500)

[LightGBM] [Info] Number of positive: 48086, number of negative: 47856
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.021169 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 1107
[LightGBM] [Info] Number of data points in the train set: 95942, number of used features: 41
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.501199 -> initscore=0.004795
[LightGBM] [Info] Start training from score 0.004795


In [237]:
f1_score(yte, model.predict(Xte.drop(id_col, axis=1)) > 0.5)

0.5618007224033125

## Генерация ответа

In [238]:
X_test_transformed = preprocessor.transform(X_test_fe)
X_test_transformed_df = pd.DataFrame(
    X_test_transformed,
    columns=preprocessor.get_feature_names_out()
)

In [241]:
y_pred = model.predict(X_test_transformed_df.drop(id_col, axis=1)) > 0.5
y_pred

array([False, False,  True, ...,  True, False, False])

In [243]:
submission = pd.DataFrame({
    'id': X_test_transformed_df[id_col].astype(int),
    'flag': y_pred
})
submission

,id,flag
0,532674,False
1,1048835,False
2,184294,True
3,1075748,False
4,2034965,True
...,...,...
456847,1502480,False
456848,1469413,True
456849,263710,True
456850,1119056,False


In [244]:
# ---------- Сохраняем submission ----------
submission.to_csv("submission.csv", index=False)
print("Saved submission.csv")

Saved submission.csv
